<a href="https://colab.research.google.com/github/choycoy/final_project/blob/main/recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
q_df = pd.read_csv("/content/drive/My Drive/final/questions.csv",sep=",")

In [ ]:
q_df['question_id'] = q_df['question_id'].str.replace('q', '')
q_df

,question_id,bundle_id,explanation_id,correct_answer,part,tags,deployed_at
0,1,b1,e1,b,1,1;2;179;181,1558093217098
1,2,b2,e2,a,1,15;2;182,1558093219720
2,3,b3,e3,b,1,14;2;179;183,1558093222784
3,4,b4,e4,b,1,9;2;179;184,1558093225357
4,5,b5,e5,c,1,8;2;179;181,1558093228439
...,...,...,...,...,...,...,...
13164,18139,b12202,e12202,b,2,24;26;183;182,1571733814684
13165,18140,b12203,e12203,a,2,24;33;183;182,1571733815331
13166,18141,b12204,e12204,a,2,24;26;183;182,1571733815951
13167,18142,b12205,e12205,a,2,24;26;183;182,1571733816585


In [ ]:
df1 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__1.csv",sep=",")
df2 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__2.csv",sep=",")
df3 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__3.csv",sep=",")
df4 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__4.csv",sep=",")
df5 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__5.csv",sep=",")
df6 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__6.csv",sep=",")
df7 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__7.csv",sep=",")
df8 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__8.csv",sep=",")
df9 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__9.csv",sep=",")
df10 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__10.csv",sep=",")
df11 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__11.csv",sep=",")
df12 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__12.csv",sep=",")
df13 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__13.csv",sep=",")
df14 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__14.csv",sep=",")
df15 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__15.csv",sep=",")
df16 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__16.csv",sep=",")
df17 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__17.csv",sep=",")
df18 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__18.csv",sep=",")
df19 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__19.csv",sep=",")
df20 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__20.csv",sep=",")
df21 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__21.csv",sep=",")
df22 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__22.csv",sep=",")
df23 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__23.csv",sep=",")
df24 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__24.csv",sep=",")
df25 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__25.csv",sep=",")
df26 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__26.csv",sep=",")
df27 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__27.csv",sep=",")
df28 = pd.read_csv("/content/drive/My Drive/final/combined_with_filename__28.csv",sep=",")


df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8,df9, df10, df11, df12, df13, df14, df15, df16, df17, df18, df19, df20,df21, df22,df23, df24, df25, df26, df27, df28])

In [ ]:
# Convert the 'question_id' column in df1 and df2 to the same data type (e.g., int64)
df['question_id'] = df['question_id'].astype('int64')
q_df['question_id'] = q_df['question_id'].astype('int64')

# Merge the dataframes on 'question_id'
merged_df = pd.merge(df, q_df[['question_id', 'correct_answer']], on='question_id', how='left')

# Create the 'is_correct' column based on the comparison between 'user_answer' and 'correct_answer'
merged_df['is_correct'] = (merged_df['user_answer'] == merged_df['correct_answer']).astype(int)

# Now merged_df will have a new column 'is_correct' indicating if the user_answer is correct or not
print(merged_df)


             timestamp  solving_id  question_id user_answer  elapsed_time  \
0        1565096190868           1         5012           b         38000   
1        1565096221062           2         4706           c         24000   
2        1565096293432           3         4366           b         68000   
3        1565096339668           4         4829           a         42000   
4        1565096401774           5         6528           b         59000   
...                ...         ...          ...         ...           ...   
2999995  1567427902612        3156        10020           b         20000   
2999996  1567428015542        3157        17192           a         14000   
2999997  1567428333271        3158         5418           b         15000   
2999998  1567428412553        3159        10020           c          4000   
2999999  1567431236639        3160         6366           c         31000   

         user_id correct_answer  is_correct  
0              1             

In [54]:
# Assuming you have the dataframe df containing student interactions with columns: timestamp, solving_id, question_id, user_answer, elapsed_time, user_id, and is_correct (1 for correct, 0 for incorrect).

# Sort the dataframe by timestamp to ensure the data is in chronological order
merged_df.sort_values(by=['user_id', 'timestamp'], inplace=True)

# Calculate Correctness Rate and Interaction Count for each user
merged_df['correct_count'] = merged_df.groupby('user_id')['is_correct'].cumsum()  # Cumulative sum of correct answers for each user
print(merged_df['correct_count'])

merged_df['interaction_count'] = merged_df.groupby('user_id').cumcount() + 1  # Cumulative count of interactions for each user
print(merged_df['interaction_count'])

merged_df['correctness_rate'] = merged_df['correct_count'] / merged_df['interaction_count']  # Calculate the correctness rate for each interaction

# Calculate average, minimum, and maximum elapsed time for each user
merged_df['average_elapsed_time'] = merged_df.groupby('user_id')['elapsed_time'].transform('mean')

merged_df

0            0
1            1
2            2
3            2
4            2
            ..
21667466    15
21667467    15
21667468    16
21667469    17
21667470    17
Name: correct_count, Length: 27904657, dtype: int64
0            1
1            2
2            3
3            4
4            5
            ..
21667466    28
21667467    29
21667468    30
21667469    31
21667470    32
Name: interaction_count, Length: 27904657, dtype: int64


,timestamp,solving_id,question_id,user_answer,elapsed_time,user_id,correct_answer,is_correct,correct_count,interaction_count,correctness_rate,average_elapsed_time
0,1565096190868,1,5012,b,38000,1,c,0,0,1,0.000000,49578.441774
1,1565096221062,2,4706,c,24000,1,c,1,1,2,0.500000,49578.441774
2,1565096293432,3,4366,b,68000,1,b,1,2,3,0.666667,49578.441774
3,1565096339668,4,4829,a,42000,1,c,0,2,4,0.500000,49578.441774
4,1565096401774,5,6528,b,59000,1,d,0,2,5,0.400000,49578.441774
...,...,...,...,...,...,...,...,...,...,...,...,...
21667466,1546410152934,26,5401,d,101000,52670,d,1,15,28,0.535714,123718.718750
21667467,1546453279153,27,9606,c,38000,52670,a,0,15,29,0.517241,123718.718750
21667468,1546454614609,28,6725,c,54000,52670,c,1,16,30,0.533333,123718.718750
21667469,1546518985980,29,8368,b,79000,52670,b,1,17,31,0.548387,123718.718750


In [55]:
user_correctness = merged_df.groupby('user_id')['correctness_rate'].mean()
user_correctness

user_id
1        0.713058
2        0.699210
3        0.400357
4        0.723672
5        0.605313
           ...   
52666    0.473780
52667    0.255762
52668    0.335124
52669    0.581180
52670    0.539483
Name: correctness_rate, Length: 45277, dtype: float64

In [56]:
user_elapsed_time = merged_df.groupby('user_id')['average_elapsed_time'].mean()
user_elapsed_time

user_id
1         49578.441774
2         28168.622847
3         62300.000000
4         29187.739533
5         27130.550689
             ...      
52666     31299.833333
52667     23000.000000
52668     16066.500000
52669     27857.464953
52670    123718.718750
Name: average_elapsed_time, Length: 45277, dtype: float64

In [57]:
user_metrics_df = pd.DataFrame({'user_id': user_correctness.index,
                                'correctness_rate': user_correctness.values,
                                'user_elapsed_time': user_elapsed_time.values})

In [58]:
# Sort the DataFrame in descending order based on 'elapsed_time_scaled'

# Calculate the 80th percentile (quantile) of 'elapsed_time_scaled'
top_20_percentile = user_metrics_df['user_elapsed_time'].quantile(0.3)
below_20_percentile = user_metrics_df['user_elapsed_time'].quantile(0.7)

high_achievers_threshold = 0.7
struggling_learners_threshold = 0.3

def segment_users(row):
    if row['correctness_rate'] >= high_achievers_threshold and row['user_elapsed_time'] <= top_20_percentile:
        return 'High Achievers'
    elif row['correctness_rate'] <= struggling_learners_threshold and row['user_elapsed_time'] >= below_20_percentile:
        return 'Struggling Learners'
    else:
        return 'Average Performers'

user_metrics_df['user_segment'] = user_metrics_df.apply(segment_users, axis=1)


In [59]:
pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2813030 sha256=fa932b0f483e16d13f1c1c5a33c9f64556e5e83985bb4ca2b4aa7aaf82fffd86
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [60]:
merged_df= merged_df.sort_index()
merged_df

,timestamp,solving_id,question_id,user_answer,elapsed_time,user_id,correct_answer,is_correct,correct_count,interaction_count,correctness_rate,average_elapsed_time
0,1565096190868,1,5012,b,38000,1,c,0,0,1,0.000000,49578.441774
1,1565096221062,2,4706,c,24000,1,c,1,1,2,0.500000,49578.441774
2,1565096293432,3,4366,b,68000,1,b,1,2,3,0.666667,49578.441774
3,1565096339668,4,4829,a,42000,1,c,0,2,4,0.500000,49578.441774
4,1565096401774,5,6528,b,59000,1,d,0,2,5,0.400000,49578.441774
...,...,...,...,...,...,...,...,...,...,...,...,...
27904652,1574639347000,1263,8599,a,19000,9999,a,1,1542,2185,0.705721,34690.245317
27904653,1574639368000,1264,8972,a,13000,9999,a,1,1543,2186,0.705855,34690.245317
27904654,1574639410000,1265,8812,c,34000,9999,c,1,1544,2187,0.705990,34690.245317
27904655,1574639466000,1266,8690,b,15000,9999,b,1,1545,2188,0.706124,34690.245317


In [61]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

reader = Reader(rating_scale=(0, 1))
custom_dataset = Dataset.load_from_df(merged_df[['user_id', 'question_id', 'correctness_rate']], reader)

In [62]:
trainset, testset = train_test_split(custom_dataset, test_size=0.2)

In [ ]:
from surprise import NMF

# Choose the NMF algorithm
algo = NMF()

# Train the model on the training set
algo.fit(trainset)

# Predict ratings for the test set
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 0.0795
MAE:  0.0681


In [ ]:
from surprise import SVD

# Choose the SVD algorithm
algo = SVD()

# Train the model on the training set
algo.fit(trainset)

# Predict ratings for the test set
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 0.0568
MAE:  0.0347


In [64]:
from surprise import BaselineOnly

# Choose the BaselineOnly algorithm
algo = BaselineOnly()

# Train the model on the training set
algo.fit(trainset)

# Predict ratings for the test set
predictions = algo.test(testset)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

Estimating biases using als...
RMSE: 0.0530
MAE:  0.0314


In [68]:
user_id = 3
user_segment = user_metrics_df[user_metrics_df['user_id'] == user_id]['user_segment'].values[0]

if user_segment == 'High Achievers':
    # Recommend challenging questions
    recommended_questions = [(question_id, algo.predict(user_id, question_id).est) for question_id in range(trainset.n_items) if algo.predict(user_id, question_id).est <= 0.4]
elif user_segment == 'Struggling Learners':
    # Recommend moderately challenging questions
    recommended_questions = [(question_id, algo.predict(user_id, question_id).est) for question_id in range(trainset.n_items) if algo.predict(user_id, question_id).est >= 0.6]
else:
    # Recommend easier questions or remedial content
    recommended_questions = [(question_id, algo.predict(user_id, question_id).est) for question_id in range(trainset.n_items) if 0.4 < algo.predict(user_id, question_id).est < 0.6]

# Sort recommended_questions based on the prediction score in descending order
recommended_questions.sort(key=lambda x: x[1], reverse=True)

print(f"Top 10 Detailed Recommendations for user {user_id}:")
for i, (question_id, prediction_score) in enumerate(recommended_questions[:10], 1):
    print(f"{i}. Question ID: {question_id}, Prediction Score: {prediction_score}")


Top 10 Detailed Recommendations for user 3:
1. Question ID: 8074, Prediction Score: 0.5981929086636381
2. Question ID: 6477, Prediction Score: 0.5976852682661554
3. Question ID: 933, Prediction Score: 0.5955224085879725
4. Question ID: 5759, Prediction Score: 0.5944022362145364
5. Question ID: 841, Prediction Score: 0.5936390801378653
6. Question ID: 101, Prediction Score: 0.5919802429671385
7. Question ID: 5458, Prediction Score: 0.590898867456711
8. Question ID: 1238, Prediction Score: 0.5898747829491836
9. Question ID: 541, Prediction Score: 0.5898178035506877
10. Question ID: 500, Prediction Score: 0.589797059643942
